In [ ]:
import scipy.io
import numpy as np
import pandas as pd
import scanpy as sc
import h5py

import os
import numpy as np
import pandas as pd
import scanpy as sc
import anndata 
import seaborn as sns
from scipy.stats import zscore
import matplotlib.pyplot as plt
import collections
from natsort import natsorted

from scipy import stats
from scipy import sparse
from sklearn.decomposition import PCA
from umap import UMAP

from matplotlib.colors import LinearSegmentedColormap
from statsmodels.stats.multitest import multipletests

from scroutines.config_plots import *
from scroutines import powerplots # .config_plots import *
from scroutines import pnmf
from scroutines import basicu
from scroutines.gene_modules import GeneModules  

In [ ]:
%%time

import subprocess

# run 

In [ ]:
ts = [6,8,10,12,14,17,21]
for ti in ts:
    for AvsC in ['A', 'C']:
        print(ti, AvsC)

        wkdir = f'/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results_atac/fimo_P{ti}_{AvsC}'
        f_peak_ngene = f'../P{ti}_{AvsC}_peaks_ngene.bed'
        f_genome_order = '../mm10.sorted.chrom.sizes'
        
        fin   = wkdir + '/fimo.tsv'
        
        # chec
        fout0 = wkdir + '/fimo_subset.tsv'
        fout1 = wkdir + '/fimo_subset_regionsort.bed'
        ftmp  = wkdir + '/fimo_subset_regionsort2.bed'
        fout2 = wkdir + '/fimo_subset_regionsort2_ngene.bed'

        
        # subset the files
        tfs_avc = [
            'RFX3', 'MEIS2', 'NFIB',
            'FOXP1', 'RORA', 'RORB', 'CUX1', 'SATB1', 'CAMTA1',
            ]
        command = ['grep'] 
        for tf in tfs_avc:
            command += ['-e', tf] 
        command += ['fimo.tsv']
        print(" ".join(command))
        
        with open(fout0, 'w') as f:
            subprocess.run(command, cwd=wkdir, stdout=f, stderr=subprocess.PIPE, text=True)
            
        # read and sort
        df = pd.read_csv(fout0, sep='\t', header=None)
        dfout = df.groupby([2,3,4]).first().reset_index()[[2,3,4,1]]
        dfout.to_csv(fout1, sep='\t', index=False, header=False)

        # bedtools sort -g ../mm10.sorted.chrom.sizes -i fimo_subset_regionsort.bed > fimo_subset_regionsort2.bed
        command = ['bedtools', 'sort', '-g', f_genome_order, '-i', fout1]
        with open(ftmp, 'w') as f:
            subprocess.run(command, cwd=wkdir, stdout=f, stderr=subprocess.PIPE, text=True)

        # bedtools closest -a fimo_subset_regionsort2.bed -b ../P8_A_peaks_ngene.bed > fimo_subset_regionsort_ngene.bed
        command = ['bedtools', 'closest', '-a', ftmp, '-b', f_peak_ngene]
        with open(fout2, 'w') as f:
            subprocess.run(command, cwd=wkdir, stdout=f, stderr=subprocess.PIPE, text=True)

# check results

In [ ]:
g_csms_a = ['Cntn5',  'Epha3', 'Epha6', 'Kirrel3', 'Ptprk', 'Robo1', 'Tenm1',]
g_csms_c = ['Astn2', 'Cdh12', 'Pcdh9', 'Sema6d']
g_tfs_a = ['RFX3', 'MEIS2', 'NFIB']
g_tfs_c = ['FOXP1', 'RORA', 'RORB', 'CUX1', 'SATB1', 'CAMTA1']

ts = [6,8,10,12,14,17,21]
for ti in ts:
    print(ti)
    # A-peaks that are closest to an A gene - look for A TFs binding sites
    f = f'/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results_atac/fimo_P{ti}_A/fimo_subset_regionsort2_ngene.bed'
    df = pd.read_csv(f, sep='\t', header=None)
    df_a = df[df[3].isin(g_tfs_a)]
    df_c = df[df[3].isin(g_tfs_c)]
    for g in g_csms_a:
        dfsub = df_a[df_a[11]==g]
        if len(dfsub) > 0:
            unq, cnts = np.unique(dfsub[3].values, return_counts=True)
            order = np.argsort(cnts)[::-1]
            print(g, unq[order], cnts[order])
            
        # c binding sites
        dfsub = df_c[df_c[11]==g]
        if len(dfsub) > 0:
            unq, cnts = np.unique(dfsub[3].values, return_counts=True)
            order = np.argsort(cnts)[::-1]
            print(g, unq[order], cnts[order])
            
for ti in ts:
    print(ti)
    # C-peaks that are closest to an C gene - look for C TFs binding sites
    f = f'/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results_atac/fimo_P{ti}_C/fimo_subset_regionsort2_ngene.bed'
    df = pd.read_csv(f, sep='\t', header=None)
    df_a = df[df[3].isin(g_tfs_a)]
    df_c = df[df[3].isin(g_tfs_c)]
    for g in g_csms_c:
        dfsub = df_c[df_c[11]==g]
        if len(dfsub) > 0:
            unq, cnts = np.unique(dfsub[3].values, return_counts=True)
            order = np.argsort(cnts)[::-1]
            print(g, unq[order], cnts[order])
        
        # a binding sites
        dfsub = df_a[df_a[11]==g]
        if len(dfsub) > 0:
            unq, cnts = np.unique(dfsub[3].values, return_counts=True)
            order = np.argsort(cnts)[::-1]
            print(g, unq[order], cnts[order])
